In [46]:
import cv2
from skimage import io
from matplotlib import pyplot as plt
from functions import apply_vegetative_index
import glob

ImportError: cannot import name 'apply_vegetative_index' from 'functions' (c:\Users\dusty\Documents\REU_2024\mtech_reu_plants\functions.py)

In [37]:
# Import image
img = cv2.imread('test_photos/test_image_22.jpg')

ImportError: cannot import name 'apply_vegetative_index' from 'functions' (c:\Users\dusty\Documents\REU_2024\mtech_reu_plants\functions.py)

In [3]:
# ExG_index = 2 * green - red - blue
# ExR_index = (1.4 * red - green) / green + red + blue
# ExB_index = (1.4 * blue - green) / green + red + blue

# GRVI_index = (green - red) / (green + red)

C:\Users\dusty\AppData\Local\Temp\ipykernel_17784\3280915200.py:5: RuntimeWarning: divide by zero encountered in divide
  GRVI_index = (green - red) / (green + red)
C:\Users\dusty\AppData\Local\Temp\ipykernel_17784\3280915200.py:5: RuntimeWarning: invalid value encountered in divide
  GRVI_index = (green - red) / (green + red)


In [35]:
exg_img = apply_vegetative_index(img, index_type='exg')

plt.imshow(exg_img)

NameError: name 'apply_vegetative_index' is not defined

Now try it on all of the test images

In [19]:
# Define a path to the folder that contains the photos I want to test with loops
path = 'test_photos/*'

In [30]:
# Loop through every file in the folder specified by the path
for i, file in enumerate(glob.glob(path)):
    # Read in the color image
    img = cv2.imread(file)

    # Use the excess green vegetative index on every image
    exg_img = exg(img)

    plt.imsave(f'results/vi_exg/image_{i}.jpg', exg_img)

#     # Show each image in its own window.
#     cv2.imshow(f'image {i}', exg_img)

# # Allows you to close all windows (individually, or close all with ctrl+w)
# cv2.waitKey(0)
# cv2.destroyAllWindows()